Now we have data from OpenNeuro in BIDS format, including derivatives such as preprocessed images and confounds(fMRIPrep).
You can look around in the directory to see what files are in our dataset.

In [1]:
import os
import glob
import nilearn
import json
import pandas as pd
from nilearn.glm.first_level import FirstLevelModel
import datalad.api as dl

/Users/sunjaeshim/opt/anaconda3/lib/python3.8/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '


In [2]:
bids_dir = '/Users/sunjaeshim/Documents/GitHub/BrainHack2023_fMRI_practice/ds001734/'
fmriprep_dir = bids_dir + 'derivatives/fmriprep/'

#You can change the subjects you want to analyze.
some_subjects = ['sub-001', 'sub-002', 'sub-003']
#If you want to analyze all data, you can save all subject ids in this dataset by using glob
all_files = glob.glob(bids_dir+'sub-*')
all_subjects = sorted([os.path.basename(i) for i in all_files])

We will need event files, preprocessed images files, brain mask files, and confound files in order to build our design matrix. You can create a function to get the corresponding files. Note that there are several runs per subject, so make sure you are organizing the correct files together.

In [3]:
def get_files(sub, run):
    run = str(run)
    
    event_file = glob.glob(bids_dir+sub+f'/func/*run-0{run}*.tsv')[0]
    
    func_file = glob.glob(fmriprep_dir+sub+f'/func/*task-MGT*run-0{run}*preproc.nii.gz')[0]
    
    mask_file = glob.glob(fmriprep_dir+sub+f'/func/*task-MGT*run-0{run}*brainmask.nii.gz')[0]
    
    confounds_file = glob.glob(fmriprep_dir+sub+f'/func/*task-MGT*run-0{run}*confounds*')[0]
    return event_file, func_file, mask_file, confounds_file

We will also need to know the TR (repetition time) of the scans. You can get this information from the BIDS scan json file

In [4]:
def get_tr(bids_dir):
    json_file = glob.glob(bids_dir+'*task*.json')[0]
    f = open(json_file)
    data = json.load(f)
    print(data)
    tr = data['RepetitionTime']
    return tr

You can choose which confound regressors to add to the analysis. You can take a look at the confounds output to evaluate what you would include to denoise the scans. 

More information about confound outputs from fMRIPrep: https://fmriprep.org/en/stable/outputs.html#confounds

In [5]:
def get_confounds(confounds_file):
    confounds_df = pd.read_csv(confounds_file, sep='\t')
    #These are the very basic confounds most recommend. Adding the derivatives of these may also improve
    #Newer versions of fMRIPrep gives you derivatives
    #Note if you add CompCor components, you also should use the fMRIPrep cosine regressors instead of nilearn's
    confounds_df  = confounds_df[['FramewiseDisplacement', 'X', 'Y', 'Z', 'RotX', 'RotY', 'RotZ']]
    
    return confounds_df

In [ ]:
tr = get_tr(bids_dir)
for sub in some_subjects:
    for i in range(1,5):
        event_file, func_file, mask_file, confounds_file = get_files(sub, i)
        dl.get(event_file)
        print(event_file)
        dl.get(func_file) 
        print(func_file)
        dl.get(mask_file)
        print(mask_file)
        dl.get(confounds_file)
        events = pd.read_csv(event_file, sep='\t')[['participant_response', 'onset', 'duration']]
        events.rename(columns = {'participant_response':'trial_type'}, inplace=True)
        events.dropna(inplace=True)
        confounds = get_confounds(confounds_file)
        first_level_model = FirstLevelModel(tr, hrf_model = 'spm + derivative')
        first_level_model = first_level_model.fit(func_file, events=events, confounds=confounds)
        design_matrix = first_level_model.design_matrices_[0]

In [ ]:
design_matrix